In [ ]:

import argparse
import numpy as np
import paddle
import torch
import os
from config_cvt.default import get_config
import torchvision.models as models
from cvt import generate_model
model_name = 'CvT-13-224x224-IN-1k.pth'

config=get_config('./config_cvt/config.yaml')
    

In [ ]:
def torch_to_paddle_mapping():
    mapping = [
        ('norm.weight', 'norm.weight'),
        ('norm.bias', 'norm.bias'),
        ('head.weight', 'head.weight'),
        ('head.bias','head.bias')
    ]
    num_layers=3
    for idx in range(num_layers):
        stage_prefix = f'stage{idx}.'
        layer_prefix='patch_embed.'
        prefix=stage_prefix+layer_prefix
        layer_mapping = [
            (f'{prefix}proj.weight', f'{prefix}proj.weight'),
            (f'{prefix}proj.bias',   f'{prefix}proj.bias'  ),
            (f'{prefix}norm.weight', f'{prefix}norm.weight'),
            (f'{prefix}norm.bias',   f'{prefix}norm.bias'  ),
        ]
        mapping.extend(layer_mapping)
        

    return mapping

In [ ]:
def convert(torch_model, paddle_model):
    def _set_value(th_name, pd_name, transpose=False):
        th_shape = th_params[th_name].shape
        pd_shape = tuple(pd_params[pd_name].shape) 

        print(f'**SET** {th_name} {th_shape} **TO** {pd_name} {pd_shape}')
        
        if isinstance(th_params[th_name], torch.nn.parameter.Parameter):
            value = th_params[th_name].data.numpy()
        else:
            value = th_params[th_name].numpy()

        if len(value.shape) == 2 and transpose:
            value = value.transpose((1, 0))
        pd_params[pd_name].set_value(value)

    # 1. get paddle and torch model parameters
    pd_params = {}
    th_params = {}
    for name, param in paddle_model.named_parameters():
        pd_params[name] = param
    for name, param in torch_model.named_parameters():
        th_params[name] = param

    for name, param in paddle_model.named_buffers():
        pd_params[name] = param
    for name, param in torch_model.named_buffers():
        th_params[name] = param

    # 2. get name mapping pairs
    mapping = torch_to_paddle_mapping()

    # 3. set torch param values to paddle params: may needs transpose on weights
    for th_name, pd_name in mapping:
        if th_name in th_params.keys(): # nn.Parameters
            _set_value(th_name, pd_name)
        else: # weight & bias
            th_name_w = f'{th_name}.weight'
            pd_name_w = f'{pd_name}.weight'
            _set_value(th_name_w, pd_name_w)

            if f'{th_name}.bias' in th_params.keys():
                th_name_b = f'{th_name}.bias'
                pd_name_b = f'{pd_name}.bias'
                _set_value(th_name_b, pd_name_b)

    return paddle_model


In [ ]:


paddle.set_device('cpu')
paddle_model = generate_model(config)
paddle_model.eval()
print('+++++++++++++++++++++++++++++++++++')
device = torch.device('cpu')
torch_model= models.squeezenet1_1(pretrained=True)
torch_model.load_state_dict= torch.load('CvT-13-224x224-IN-1k.pth',map_location=torch.device('cpu'))

torch_model = torch_model.to(device)
torch_model.eval()

# convert weights
paddle_model = convert(torch_model, paddle_model)

# check correctness
x = np.random.randn(2, 3, 224, 224).astype('float32')
x_paddle = paddle.to_tensor(x)
x_torch = torch.Tensor(x).to(device)

out_torch = torch_model(x_torch)
out_paddle = paddle_model(x_paddle)

out_torch = out_torch.data.cpu().numpy()
out_paddle = out_paddle.cpu().numpy()

print(out_torch.shape, out_paddle.shape)
print(out_torch[0, 0:100])
print('========================================================')
print(out_paddle[0, 0:100])

assert np.allclose(out_torch, out_paddle, atol = 1e-5)

# save weights for paddle model
model_path = os.path.join(f'./{model_name}.pdparams')
paddle.save(paddle_model.state_dict(), model_path)
print('all done')
